# Análise de sentimenos usando NLP

- O seguinte projeto tem como objetivo criar um modelo NLP classificador para analisar o humor dos usuários baseados em seus dados (tweets);
- A plataforma escolhida para essa atividade foi o X (Twitter), por meio do dataset público sentiment140 disponível no Kaggle;

# Equipe:
- Pedro Yutaro Mont Morency Nakamura;
- Lucas Carvalho dos Santos;
- Caio Jorge Da Cunha Queiroz;
- Lucas Maciel ;omes;

# 1 - Importação de bibliotecas
Inicialmente iremos importar as seguintes bibliotecas:
- *Pandas + Numnpy*: Manipulação de dados;
- *Scikit-learn*: Pré-processamento e criação de métricas do modelo;
- *NLTK*: Biblioteca para pre-processamento de texto humano (toolkit de linguagem natural);
- *matplotlib*: Criação de gráficos para a vizualização dos resultados;

In [18]:
# Analise e tratamento de dados
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
# Processamento de linguagem natural
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
# Bibliotecas para o classificador
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

O Conteudo do dataset contem os seguintes campos:

target: the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)

ids: The id of the tweet ( 2087)

date: the date of the tweet (Sat May 16 23:58:44 UTC 2009)

flag: The query (lyx). If there is no query, then this value is NO_QUERY.

user: the user that tweeted (robotickilldozr)

text: the text of the tweet (Lyx is cool)

In [3]:
# Carregando o dataset do keagle
df = pd.read_csv("tweets_data.csv", encoding="latin-1", header=None)
# Renomeando colunas conforme a documentação
df.columns = ["target", "id", "date", "flag", "user", "text"]
# Iremos usar apenas essas colunas em nosso treino
df = df[["target", "id", "text"]]
# Conversão de status
df["sentiment"] = df["target"].apply(lambda x: "positivo" if x >= 3 else "negativo")
df.head()

,target,id,text,sentiment
0,0,1467810369,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",negativo
1,0,1467810672,is upset that he can't update his Facebook by ...,negativo
2,0,1467810917,@Kenichan I dived many times for the ball. Man...,negativo
3,0,1467811184,my whole body feels itchy and like its on fire,negativo
4,0,1467811193,"@nationwideclass no, it's not behaving at all....",negativo


Como é possível observar, agora o nosso dataframe possui apenas a polaridade, o texto e o sentimento correspondente

Porém é possível que parte desse texto contenha "ruídos" como URLs, menções, emojis ou símbolos especiais. Para isso, iremos fazer uma breve limpeza usando nltk para lidar com stopwords:

Também precisamos verificar a possível existencia de tweets duplicados pelo seu ID como se segue:

In [4]:
def show_amount_dup(data):
    dup = round(sum(data.duplicated("id")/len(data)*100), 3)
    print(f"Percentual de tweets duplicados: {dup}%")

show_amount_dup(df)

Percentual de tweets duplicados: 0.105%


In [5]:
# Removendo tweets duplicados
df.drop_duplicates("id", inplace=True)
show_amount_dup(df)

Percentual de tweets duplicados: 0.0%


In [6]:
# Atualizando dataset pra usar apenas colunas relevantes
df = df.drop(columns=["id"])

In [8]:
nltk.download("stopwords")
nltk.download("punkt")
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pedro\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pedro\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\pedro\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [16]:
stop_words = set(stopwords.words("english"))

reURL = r"http\S+|www\S+|https\S+"
reMention = r"@\w+"
reSpecChars_and_Nums = r"[^A-Za-z\s]"

def preprocess_text(txt):
    # remove urls
    txt = re.sub(reURL, "", txt)
    txt = re.sub(reMention, "", txt)
    txt = re.sub(reSpecChars_and_Nums, "", txt))
    # tokenizar e remover stop words
    tokens = word_tokenize(txt.lower())
    tokens = [word for word in tokens if word not in stop_words]
    return " ".join(tokens)

In [ ]:
df["text"] = df["text"].apply(preprocess_text)

# 4. Divisão dos dados
Com os dados ja tratados, iremos dividi-los em dados de treino e teste:

In [ ]:
X = df["text"]
y = df["sentiment"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)